## 🏎️ F1 Declarative Pipeline

This notebook demonstrates Declarative Pipeline concepts, but the **actual runnable pipeline** is located in:

### 📂 **`06_Formula1_Declarative_Pipeline`**

**👉 Navigate there now to run the pre-built F1 qualifying data pipeline!**

---

![](./Images/06_Formula1_Declarative_Pipeline folder.png "06_Formula1_Declarative_Pipeline folder.png")

## 🚀 Building Your Declarative Pipeline

**Step-by-Step Process:**

### 1️⃣ Get the Pipeline Code
Navigate to the **`06_Formula1_Declarative_Pipeline/transformations/`** folder and open **`formula1_sdp.py`**

This file contains the complete Declarative Pipeline code that you'll copy into your new pipeline.

![](./Images/06_Formula1_Declarative_Pipeline.png "06_Formula1_Declarative_Pipeline.png")

### 2️⃣ Create New ETL Pipeline
1. **Navigate up one level** to the main notebooks folder
2. **Click "Create"** → **"ETL Pipeline"**

![](./Images/Create ETL Pipeline Navigation.png "Create ETL Pipeline Navigation.png")

3. **Name your pipeline:** `Formula1 Declarative Pipeline` (or your preferred name)
4. **Select "Start with empty file"**
5. **Choose "Python"** as the language

![](./Images/Declarative Pipeline Plus Empty File.png "Declarative Pipeline Plus Empty File.png")

### 3️⃣ Add Pipeline Code
1. **Copy the code** from `formula1_sdp.py` 
2. **Paste it** into your new empty pipeline file
3. **Click "Run Pipeline"** to execute

![](./Images/Declarative Pipeline DAG.png "Declarative Pipeline DAG.png")

---

## 🎯 What This Pipeline Does

**🥉 Bronze Layer:**
- Incrementally ingests F1 sprint qualifying CSV files using Auto Loader
- Applies data quality checks (no rescued data, valid track/driver)

**🔄 Change Data Capture:**
- Creates streaming table for sprint qualifying results
- Uses Track + Driver as composite keys
- Sequences updates by Q1 qualifying times

## 🚀 Next Steps

Ready to explore AI-powered features and advanced analytics?

### Immediate Actions:
1. **🔄 Create Your Declarative Pipeline:**
   - Go to Workflows → Declarative Pipelines → Create Pipeline
   - Use this notebook as the source
   - Configure with Serverless compute

2. **📊 Monitor Pipeline Execution:**
   - Watch automatic dependency resolution
   - Check data quality expectation results
   - Explore generated lineage graphs

3. **➡️ Next Notebook:** [07_SQL_Editor.sql](07_SQL_Editor.sql)
   - Explore SQL analytics and visualization capabilities
   - Build interactive F1 dashboards and reports

### 🎯 Best Practices Checklist:
- ✅ **Start simple** with basic Bronze → Silver → Gold
- ✅ **Add expectations gradually** as you understand your data
- ✅ **Use descriptive table names** for clarity
- ✅ **Document transformations** with comments
- ✅ **Monitor data quality** trends over time
- ✅ **Test expectations** before production deployment

### 💡 Pro Tips:
- **🔧 Start with `@dlt.expect()`** to understand data patterns
- **📊 Use pipeline dashboards** for quality monitoring
- **⚡ Leverage Serverless** for cost-effective execution
- **🔄 Design for incremental processing** from day one

**🔄 Your data pipelines are now production-ready! 🚀**